# Australian COVID Vaccinations and Work (circa mid 2022)
This example explores the mandatory COVID vaccination regime in place in the middle of 2022. Three Australian states: New South Wales, Victoria, and Western Australia are covered in this example.

The sigma complete relation ```covid_vaccinations_and_work``` which contains the relevant rules is defined in [MiniZinc](https://www.minizinc.org/). Here is the definition.

In [1]:
%reload_ext jetisu.query_idr_magic

In [2]:
%%jetisu_show
covid_vaccinations_and_work

```

enum work_location_enum = { new_south_wales, victoria, western_australia};
enum work_sector_enum = {education, aged_care, healthcare, custodial, emergency_services, disability_services,  other };

predicate covid_vaccinations_and_work(
    var work_location_enum: work_location,
    var work_sector_enum: work_sector,
    var bool: specialist_school,
    var bool: aged_care_facility,
    var bool: disability_worker_in_school,
    var bool: private_home_only,
    var bool: nsw_health_worker,
    var bool: covid_vaccination_work_mandatory,
    var int: covid_vaccination_work_recommended_doses
    ) =
let {
constraint (covid_vaccination_work_recommended_doses > 0) <-> covid_vaccination_work_mandatory;
constraint disability_worker_in_school -> (work_sector = education);
constraint specialist_school -> (work_sector = education);
constraint aged_care_facility -> (work_sector = aged_care);
constraint nsw_health_worker -> (work_sector = healthcare /\ work_location = new_south_wales);
constraint covid_vaccination_work_recommended_doses =
  if work_location = new_south_wales then
    if work_sector = education then
      if disability_worker_in_school then
        3
      else
        0
      endif
    elseif work_sector = aged_care then
      if aged_care_facility then
        3
      else
        2
      endif
    elseif work_sector = disability_services then
      3
    elseif work_sector = healthcare then
      if nsw_health_worker then
        3
      else
        0
      endif
    else
      0
    endif
  elseif  work_location = victoria then
    if work_sector = education then
      if specialist_school then
        3
      else
        0
      endif
    elseif work_sector = aged_care then
      if aged_care_facility then
        3
      else
        0
      endif
    elseif work_sector = custodial then
      3
    elseif work_sector = emergency_services then
      3
    elseif work_sector = disability_services then
      3
    elseif work_sector = healthcare then
      3
    elseif work_sector = other then
      0
    else
      0
    endif
  elseif  work_location = western_australia then
    if work_sector = aged_care then
      if private_home_only then
        0
      else
        3
      endif
    elseif work_sector = disability_services then
      if private_home_only then
        0
      else
        3
      endif
    elseif work_sector = healthcare then
      if private_home_only then
        0
      else
        3
      endif
    elseif work_sector = other then
      0
    else
      0
    endif
  else
    0
  endif;
} in true;
```

And here are some sample queries.

Which roles within New South Wales require 3 COVID vaccinations?

In [3]:
%%jetisu_query
select distinct work_location,
work_sector,
specialist_school,
aged_care_facility,
disability_worker_in_school,
nsw_health_worker from covid_vaccinations_and_work where work_location = 'new_south_wales' and covid_vaccination_work_recommended_doses=3 order by work_sector;

|work_location|work_sector|specialist_school|aged_care_facility|disability_worker_in_school|nsw_health_worker|
|----|----|----|----|----|----|
|new_south_wales|aged_care|False|True|False|False|
|new_south_wales|disability_services|False|False|False|False|
|new_south_wales|education|False|False|True|False|
|new_south_wales|education|True|False|True|False|
|new_south_wales|healthcare|False|False|False|True|

And in Western Australia, which roles require 3 COVID vaccinations?

In [4]:
%%jetisu_query
select distinct work_location,
work_sector,
specialist_school,
aged_care_facility,
disability_worker_in_school,
private_home_only from covid_vaccinations_and_work where work_location = 'western_australia' and covid_vaccination_work_recommended_doses=3 order by work_location;

|work_location|work_sector|specialist_school|aged_care_facility|disability_worker_in_school|private_home_only|
|----|----|----|----|----|----|
|western_australia|aged_care|False|False|False|False|
|western_australia|aged_care|False|True|False|False|
|western_australia|disability_services|False|False|False|False|
|western_australia|healthcare|False|False|False|False|

What is the requirement for emergency services workers across the three States?

In [5]:
%%jetisu_query
select distinct work_location, covid_vaccination_work_recommended_doses from covid_vaccinations_and_work where work_sector = 'emergency_services';

|work_location|covid_vaccination_work_recommended_doses|
|----|----|
|new_south_wales|0|
|victoria|3|
|western_australia|0|

If I am a NSW Health Worker, what vaccinations am I required to have?  Notice here that the work location and work sector of the person are correctly inferred.

In [6]:
%%jetisu_query
select distinct nsw_health_worker, work_location, work_sector, covid_vaccination_work_recommended_doses from covid_vaccinations_and_work where nsw_health_worker;

|nsw_health_worker|work_location|work_sector|covid_vaccination_work_recommended_doses|
|----|----|----|----|
|True|new_south_wales|healthcare|3|

## Test Cases
In addition to these examples, the ```covid_vaccinations_and_work``` intensionally defined relation passes an additional 23 tests in ```test_covid_vaccinations_and_work.py```.

## Next step
You can edit and re-run this example workbook by following the instructions here:
[How to run the example notebooks](https://github.com/DavidPratten/jetisu/blob/main/docs/run_notebooks.md)